### **homework 4: self-attention**
- classify the speakers of given features
- main goal: learn how to use transformer
- baselines:
  - easy: run sample code and know how to use transformer
  - medium: know how to adjust parameters of transformer
  - hard: construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer

- other links
  - kaggle: [link](https://www.kaggle.com/t/859c9ca9ede14fdea841be627c412322)
  - slide: [link](https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW04/HW04.pdf)
  - data: [link](https://drive.google.com/file/d/1gaFy8RaQVUEXo2n0peCBR5gYKCB-mNHc/view?usp=sharing)

### **data**

#### **dataset**

- args:
  - data_dir: the path to the data directory
  - metadata_path: the path to the metadata
  - segment_len: the length of audio segment for training


- the architecture of data directory <br>
  - data directory <br>
  |---- metadata.json <br>
  |---- testdata.json <br>
  |---- mapping.json <br>
  |---- uttr-{random string}.pt <br>

- the information in metadata
  - "n_mels": the dimention of mel-spectrogram
  - "speakers": a dictionary. 
    - key: speaker ids.
    - value: "feature_path" and "mel_len"

for efficiency, we segment the mel-spectrograms into segments in the traing step

In [ ]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

class myDataset(Dataset):
  def __init__(self, data_dir, segment_len=128):
    self.data_dir = data_dir
    self.segment_len = segment_len

    # load the mapping from speaker neme to their corresponding id
    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())
    self.speaker2id = mapping["speaker2id"]

    # load metadata of training data
    metadata_path = Path(data_dir) / "metadata.json"
    metadata = json.load(open(metadata_path))["speakers"]

    # get the total number of speaker
    self.speaker_num = len(metadata.keys())
    self.data = []
    for speaker in metadata.keys():
      for utterances in metadata[speaker]:
        self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    feat_path, speaker = self.data[index]
    # load preprocessed mel-spectrogram
    mel = torch.load(os.path.join(self.data_dir, feat_path), weights_only=False)
    
    # segmemt mel-spectrogram into "segment_len" frames
    if len(mel) > self.segment_len:
      # randomly get the starting point of the segment
      start = random.randint(0, len(mel) - self.segment_len)
      # get a segment with "segment_len" frames
      mel = torch.FloatTensor(mel[start:start + self.segment_len])
    else:
      mel = torch.FloatTensor(mel)
    # turn the speaker id into long for computing loss later
    speaker = torch.FloatTensor([speaker]).long()
    return mel, speaker

  def get_speaker_number(self):
    return self.speaker_num

#### **dataloader**
- split dataset into training dataset(90%) and validation dataset(10%)
- create dataloader to iterate the data

In [2]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
  # process features within a batch
  """collate a batch of data"""
  mel, speaker = zip(*batch)
  # because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value
  # mel: (batch size, length, 40)
  return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
  """generate dataloader"""
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)

  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=True,
    collate_fn=collate_batch,
  )

  return train_loader, valid_loader, speaker_num


### **model**
- TransformerEncoderLayer:
  - base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - parameters:
    - d_model: the number of expected features of the input (required) <br>
    - nhead: the number of heads of the multiheadattention models (required) <br>
    - dim_feedforward: the dimension of the feedforward network model (default=2048) <br>
    - dropout: the dropout value (default=0.1) <br>
    - activation: the activation function of intermediate layer, relu or gelu (default=relu) <br>

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required) <br>
    - num_layers: the number of sub-encoder-layers in the encoder (required) <br>
    - norm: the layer normalization component (optional) <br>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
  def __init__(self, d_model=80, n_spks=600, dropout=0.1):
    super().__init__()
    # project the dimension of features from that of input into d_model
    self.prenet = nn.Linear(40, d_model)
    # TODO:
    #   change Transformer to Conformer.
    #   https://arxiv.org/abs/2005.08100
    self.encoder_layer = nn.TransformerEncoderLayer(
      d_model=d_model, dim_feedforward=256, nhead=2
    )
    # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

    # project the the dimension of features from d_model into speaker nums
    self.pred_layer = nn.Sequential(
      nn.Linear(d_model, d_model),
      nn.ReLU(),
      nn.Linear(d_model, n_spks),
    )

  def forward(self, mels):
    """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
    # out: (batch size, length, d_model)
    out = self.prenet(mels)
    # out: (length, batch size, d_model)
    out = out.permute(1, 0, 2)
    # the encoder layer expect features in the shape of (length, batch size, d_model)
    out = self.encoder_layer(out)
    # out: (batch size, length, d_model)
    out = out.transpose(0, 1)
    # mean pooling
    stats = out.mean(dim=1)

    # out: (batch, n_spks)
    out = self.pred_layer(stats)
    return out

### **learning rate schedule**
- for transformer architecture, the design of learning rate schedule is different from that of CNN
- previous works show that the warmup of learning rate is useful for training models with transformer architectures
- the warmup schedule
  - set learning rate to 0 in the beginning
  - the learning rate increases linearly from 0 to initial learning rate during warmup period

In [4]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR

def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,
  num_training_steps: int,
  num_cycles: float = 0.5,
  last_epoch: int = -1,
):
  """
  create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer

  args:
    optimizer (:class:`~torch.optim.Optimizer`):
      the optimizer for which to schedule the learning rate
    num_warmup_steps (:obj:`int`):
      the number of steps for the warmup phase
    num_training_steps (:obj:`int`):
      the total number of training steps
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      the number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine)
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training

  return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule
  """

  def lr_lambda(current_step):
    # warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)


### **model function**
- model forward function

In [5]:
import torch


def model_fn(batch, model, criterion, device):
  """forward a batch through the model"""

  mels, labels = batch
  mels = mels.to(device)
  labels = labels.to(device)

  outs = model(mels)

  loss = criterion(outs, labels)

  # get the speaker id with highest probability
  preds = outs.argmax(1)
  # compute accuracy
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy


### **validate**
- calculate accuracy of the validation set

In [6]:
from tqdm import tqdm
import torch

def valid(dataloader, model, criterion, device):
  """validate on validation set"""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="valid", unit=" uttr")

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.2f}",
      accuracy=f"{running_accuracy / (i+1):.2f}",
    )

  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)

### **main function**

In [ ]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
  """arguments"""
  config = {
    "data_dir": "./Dataset",
    "save_path": "model.ckpt",
    "batch_size": 32,
    "n_workers": 8,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 10000,
    "total_steps": 70000,
  }

  return config


def main(
  data_dir,
  save_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[info]: use {device} now!")

  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  train_iterator = iter(train_loader)
  print(f"[info]: finish loading data!", flush=True)

  model = Classifier(n_spks=speaker_num).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = AdamW(model.parameters(), lr=1e-3)
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[info]: finish creating model!", flush=True)

  best_accuracy = -1.0
  best_state_dict = None

  pbar = tqdm(total=valid_steps, ncols=0, desc="train", unit=" step")

  for step in range(total_steps):
    # get data
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)

    loss, accuracy = model_fn(batch, model, criterion, device)
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()

    # updata model
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    # log
    pbar.update()
    pbar.set_postfix(
      loss=f"{batch_loss:.2f}",
      accuracy=f"{batch_accuracy:.2f}",
      step=step + 1,
    )

    # do validation
    if (step + 1) % valid_steps == 0:
      pbar.close()

      valid_accuracy = valid(valid_loader, model, criterion, device)

      # keep the best model
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()

      pbar = tqdm(total=valid_steps, ncols=0, desc="train", unit=" step")

    # save the best model so far
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      torch.save(best_state_dict, save_path)
      pbar.write(f"step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

  pbar.close()


if __name__ == "__main__":
  main(**parse_args())

### **inference**

#### **dataset of inference**

In [42]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset

class InferenceDataset(Dataset):
  def __init__(self, data_dir):
    testdata_path = Path(data_dir) / "testdata.json"
    metadata = json.load(testdata_path.open())
    self.data_dir = data_dir
    self.data = metadata["utterances"]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    utterance = self.data[index]
    feat_path = utterance["feature_path"]
    mel = torch.load(os.path.join(self.data_dir, feat_path), weights_only=False)

    return feat_path, mel


def inference_collate_batch(batch):
  """collate a batch of data"""
  feat_paths, mels = zip(*batch)
  return feat_paths, torch.stack(mels)


#### **main funcrion of inference**

In [ ]:
import json
import csv
from pathlib import Path

import torch
from torch.utils.data import DataLoader

from tqdm import tqdm

def parse_args():
  """arguments"""
  config = {
    "data_dir": "./Dataset",
    "model_path": "./model.ckpt",
    "output_path": "./output.csv"
  }

  return config

def main(data_dir, model_path, output_path):
  """main function"""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[info]: use {device} now!")

  mapping_path = Path(data_dir) / "mapping.json"
  mapping = json.load(mapping_path.open())

  dataset = InferenceDataset(data_dir)
  dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=8,
    collate_fn=inference_collate_batch,
  )
  print(f"[info]: finish loading data!", flush=True)

  speaker_num = len(mapping["id2speaker"])
  model = Classifier(n_spks=speaker_num).to(device)
  model.load_state_dict(torch.load(model_path,  weights_only=False))
  model.eval()
  print(f"[info]: finish creating model!", flush=True)

  results = [["Id", "Category"]]
  for feat_paths, mels in tqdm(dataloader):
    with torch.no_grad():
      mels = mels.to(device)
      outs = model(mels)
      preds = outs.argmax(1).cpu().numpy()
      for feat_path, pred in zip(feat_paths, preds):
        results.append([feat_path, mapping["id2speaker"][str(pred)]])

  with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)


if __name__ == "__main__":
  main(**parse_args())

### **debug**

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
  def __init__(self):
    self.data = [(torch.tensor([1, 2]), 0),
                  (torch.tensor([3, 4, 5]), 1),
                  (torch.tensor([6]), 2)]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

def my_collate_fn(batch):
  data = [item[0] for item in batch]
  labels = [item[1] for item in batch]

  padded_data = torch.nn.utils.rnn.pad_sequence(data, batch_first=True)
  
  return padded_data, torch.tensor(labels)

dataset = MyDataset()
data_loader = DataLoader(dataset, batch_size=2, collate_fn=my_collate_fn)

for batch in data_loader:
  padded_data, labels = batch
  print("padded data:", padded_data)
  print("labels:", labels)

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

seq1 = torch.tensor([[1, 2], [3, 4]])  # 2x2
seq2 = torch.tensor([[5, 6]])          # 1x2
seq3 = torch.tensor([[7, 8], [9, 10], [11, 12]])  # 3x2

sequences = [seq1, seq2, seq3]

padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=-20)

print(padded_sequences)

In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
src = torch.rand(10, 32, 512)
out = encoder_layer(src)
out.shape

In [ ]:
import os
import torch

data_dir = "./Dataset"

feat_path = "uttr-b6870e11053f4184aa5688e4353026b6.pt"

mel = torch.load(os.path.join(data_dir, feat_path), weights_only=False)

print(mel.shape)
print(len(mel))